In [494]:
from polar_codes.polar_code import PolarCode
from polar_codes.channels.bpsk_awgn_channel import BpskAwgnChannel
import numpy as np

In [495]:
channel = BpskAwgnChannel(3)

In [496]:
ebn0 = np.linspace(0, 1.5, 100)

In [497]:
10*np.log10(ebn0)

<ipython-input-497-ba153d25508f>:1: RuntimeWarning: divide by zero encountered in log10
  10*np.log10(ebn0)


array([        -inf, -18.19543936, -15.1851394 , -13.42422681,
       -12.17483944, -11.20573931, -10.41392685,  -9.74445896,
        -9.16453949,  -8.65301426,  -8.19543936,  -7.7815125 ,
        -7.40362689,  -7.05600583,  -6.734159  ,  -6.43452676,
        -6.15423953,  -5.89095014,  -5.6427143 ,  -5.40790335,
        -5.1851394 ,  -4.97324641,  -4.77121255,  -4.578161  ,
        -4.39332694,  -4.21603927,  -4.04570588,  -3.88180171,
        -3.72385904,  -3.57145938,  -3.42422681,  -3.28182242,
        -3.14393957,  -3.01029996,  -2.88065018,  -2.75475891,
        -2.63241435,  -2.51342211,  -2.39760339,  -2.28479329,
        -2.17483944,  -2.06760079,  -1.96294645,  -1.8607548 ,
        -1.76091259,  -1.66331422,  -1.56786104,  -1.47446078,
        -1.38302698,  -1.29347856,  -1.20573931,  -1.11973759,
        -1.03540592,  -0.95268066,  -0.87150176,  -0.79181246,
        -0.71355909,  -0.6366908 ,  -0.56115942,  -0.48691924,
        -0.41392685,  -0.34214101,  -0.27152246,  -0.20

In [498]:
n = 9
K = 256
code = PolarCode(n=n, K=K,
                 construction_method='PW',
                 channel=channel, CRC_len=24)
R = K / 2**n 

115 информационных бит для передачи

In [499]:
u_message = np.asarray([0 if np.random.random_sample() > 0.5 else 1 for _ in range(0, K)], dtype='uint8')
u_message

array([1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1,
       1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1,
       0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1,
       1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1,
       0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1,
       0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0,
       0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0,
       0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1,
       0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0,
       0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0], dtype=uint8)

In [500]:
x_message = code.encode(u_message)

In [501]:
to_message = channel.modulate(x_message)
from_message = channel.transmit(to_message)
y_message = channel.demodulate(from_message)
y_message

array([0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1,
       1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1,
       0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0,
       0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1,
       0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1,
       0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0,
       0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0,
       1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1,
       0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1,
       0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0,
       0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1,
       0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0,
       0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1,
       1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0,

In [502]:
%%time
scl_u_est_message = code.decode(y_message, decoding_method='SCL', list_size=32)
#scl_u_est_message

CPU times: user 1.22 s, sys: 12.2 ms, total: 1.23 s
Wall time: 1.29 s


In [503]:
code.get_message_info_bits(scl_u_est_message)

array([1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1,
       1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1,
       0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1,
       1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1,
       0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1,
       0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0,
       0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0,
       0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1,
       0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0,
       0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0], dtype=uint8)

In [504]:
class rec_dec:
    def __init__(self, code, y_message, i, u_est, is_calc_llr, llr_array, pm=0, PM=0):
        self.u_est = u_est
        self.is_calc_llr = is_calc_llr
        self.llr_array = llr_array
        self.y_message = y_message
        self.pm = pm #Iddooose metric
        self.layer = i
        self.code = code
        self.PM = PM #paper_metric
    
    def split(self):
        #Создаем 2 копии этого же instance c разными параметрами
        u_right = self.u_est.copy()
        u_opp = self.u_est.copy()
        
        self.llr = code._fast_llr(self.layer, self.y_message, self.u_est[:self.layer], self.llr_array, self.is_calc_llr) 
        
        u_right[self.layer] = 0 if self.llr > 0 else 1
        u_opp[self.layer] = 1 - u_right[self.layer]
        
        is_calc_llr_copy = self.is_calc_llr.copy()
        llr_array_copy = self.llr_array.copy()
        
        pm_copy = self.pm
        llr_copy = self.llr
        
        layer_copy = self.layer
        
        #calc 2 PM and provide them
        PM_right = np.log(1+np.exp(-(1-2*u_right[self.layer])*self.llr))
        new_PM_right = self.PM+PM_right
        
        PM_opp = np.log(1+np.exp(-(1-2*u_opp[self.layer])*self.llr)) 
        new_PM_opp = self.PM+PM_opp
        
        right = rec_dec(self.code, self.y_message, layer_copy+1, u_right, is_calc_llr_copy, llr_array_copy, pm_copy, new_PM_right)
        opp = rec_dec(self.code, self.y_message, layer_copy+1, u_opp, is_calc_llr_copy, llr_array_copy, pm_copy+np.abs(llr_copy), new_PM_opp)
        
        return [right, opp]
    
    def meet_frozen(self):
        self.llr = code._fast_llr(self.layer, self.y_message, self.u_est[:self.layer], self.llr_array, self.is_calc_llr) 
        self.u_est[self.layer] = 0
        self.pm += np.abs(self.llr) if (self.llr < 0) else 0 
        self.PM += np.log(1+np.exp(-(1-2*self.u_est[self.layer])*self.llr)) 
        self.layer +=1
    
    def get_pm(self):
        return self.pm
    
    def get_PM(self):
        return self.PM
    
    def get_is_calc(self):
        return self.is_calc_llr
    
    def get_l(self):
        return self.layer
    
    def get_u_est(self):
        return self.u_est
    
    def get_llr_array(self):
        return self.llr_array

In [505]:
def scl_dec(code, y_message, list_size=32, train=None): #we should decide here whether to by Indooose Metric or paper's
    u_est = np.full(code._N, -1)
    is_calc_llr = [False] * code._N * (code._n + 1)
    llr_array = np.full(code._N * (code._n + 1), 0.0, dtype=np.longfloat)
    dec_array = [] # will store L decoders always
    a = rec_dec(code, y_message, 0, u_est, is_calc_llr, llr_array)
    dec_array.append(a)
    M = np.zeros(code._N)
    
    bit_for_flip_train = None
    
    for i in range(code._N): #code._N
        
        if i in code._frozen_bits_positions:
            for elem in dec_array:
                 elem.meet_frozen()
                
        else:
            new_arr = []
            for elem in dec_array:
                temp = elem.split()
                new_arr.append(temp[0])
                new_arr.append(temp[1])
            dec_array = new_arr.copy()
            
        
        if len(dec_array) > list_size:
            dec_array.sort(key=lambda x: x.get_PM())
            M[i] = np.sum([x.get_PM() for x in dec_array[list_size:]]) - np.sum([x.get_PM() for x in dec_array[:list_size]])
            
        else:
            M[i] = - np.sum([x.get_PM() for x in dec_array[:list_size]])
        
        if len(dec_array) > list_size:      #pruning from 2L to L decoders
            dec_array.sort(key=lambda x: x.get_PM()) # by paper (mb create new decoder parametr)
            
            if (train is not None) and (bit_for_flip_train is None): #should save i, if right solution among discarded
                
                for inst in dec_array[list_size:]:
                    if (train[:i+1] == inst.get_u_est()[:i+1]).all():
                        bit_for_flip_train = i      
                
            dec_array = dec_array[:list_size]#.copy() # delete discarded paths
            
    return dec_array, M, bit_for_flip_train

In [506]:
%%time
c, M, bit = scl_dec(code, y_message, 32, code.extend_info_bits(u_message))

CPU times: user 2.1 s, sys: 340 ms, total: 2.44 s
Wall time: 2.48 s


In [507]:
bit

In [508]:
[x.get_PM() for x in c]

[64.411665534160718764,
 86.24502356078302849,
 86.245023560946472596,
 86.245023560946472596,
 86.24502356096006824,
 86.24502356096006824,
 86.245023561206244904,
 86.245023561206244904,
 86.2450235618213306,
 86.2450235618213306,
 86.2450235618213306,
 86.2450235618213306,
 86.24502356185357986,
 86.24502356185357986,
 86.24502356185357986,
 86.24502356185357986,
 86.24502356185368173,
 86.24502356185368173,
 86.24502356185368173,
 86.24502356185368173,
 86.2450235618538343,
 86.2450235618538343,
 91.04959125365938632,
 91.04959125365938632,
 93.753857910136212526,
 93.753857910136212526,
 93.753857910136212526,
 93.753857910136212526,
 93.75385791039544681,
 93.75385791039544681,
 93.75385791039544681,
 93.75385791039544681]

In [509]:
(c[0].get_llr_array() == c[1].get_llr_array()).all()

False

In [510]:
for d in c:
    print(np.abs(u_message - code.get_message_info_bits(d.get_u_est())).sum())

0
6
8
8
4
3
2
4
4
10
4
4
2
4
4
9
8
12
8
20
4
8
8
6
6
4
10
8
6
4
10
8


In [511]:
for d in c:
    print((code._calculate_CRC(u_message) == code._calculate_CRC(code.get_message_info_bits(d.get_u_est()))).all())

True
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False


In [512]:
len(M)

512

In [522]:
a = [1,2,3]
b = a
a[0] = 10

In [523]:
a

[10, 2, 3]

In [524]:
b

[10, 2, 3]